In [1]:
import pandas as pd 


In [33]:
sentiment=pd.read_csv('sentiment_data.csv')
price=pd.read_csv('apple_stock.csv')

In [34]:
price=price.dropna()

In [35]:
price['Date'] = pd.to_datetime(price['Date'])


In [36]:

sentiment['time_published'] = pd.to_datetime(sentiment['time_published'])

sentiment['Date'] = sentiment['time_published'].dt.date

daily_sentiment = sentiment.groupby('Date').agg(
    overall_sentiment_score=('overall_sentiment_score', 'mean'),
    textblob_polarity=('textblob_polarity', 'mean'),
    textblob_subjectivity=('textblob_subjectivity', 'mean'),
    transformers_sentiment_score=('transformers_sentiment_score', 'mean'),
).reset_index()

price['Date'] = pd.to_datetime(price['Date']).dt.date

combined_data = pd.merge(price, daily_sentiment, on='Date', how='left')

combined_data = combined_data.fillna(0)
combined_data['Close'] = pd.to_numeric(combined_data['Close'], errors='coerce')

combined_data['Price Change'] = combined_data['Close'].shift(-1) - combined_data['Close']
combined_data['Price Direction'] = (combined_data['Price Change'] > 0).astype(int)



In [37]:
combined_data.shape

(488, 13)

In [38]:
import numpy as np


combined_data['SMA_20'] = combined_data['Close'].rolling(window=20).mean()

combined_data['EMA_20'] = combined_data['Close'].ewm(span=20, adjust=False).mean()

ema12 = combined_data['Close'].ewm(span=12, adjust=False).mean()
ema26 = combined_data['Close'].ewm(span=26, adjust=False).mean()
combined_data['MACD'] = ema12 - ema26

combined_data['Signal_Line'] = combined_data['MACD'].ewm(span=9, adjust=False).mean()

delta = combined_data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()

rs = gain / loss
combined_data['RSI_14'] = 100 - (100 / (1 + rs))



In [39]:
combined_data.columns

Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       'overall_sentiment_score', 'textblob_polarity', 'textblob_subjectivity',
       'transformers_sentiment_score', 'Price Change', 'Price Direction',
       'SMA_20', 'EMA_20', 'MACD', 'Signal_Line', 'RSI_14'],
      dtype='object')

In [40]:
combined_data=combined_data.dropna()

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler


X = combined_data[['Price Change', 'SMA_20', 'EMA_20', 'MACD', 'Signal_Line', 'RSI_14',
                'overall_sentiment_score', 'textblob_polarity', 'textblob_subjectivity', 'transformers_sentiment_score']]

y = combined_data['Price Direction']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [46]:
from sklearn.linear_model import LogisticRegression

#  logistic regression with L2 regularization
model = LogisticRegression(C=1.0, solver='lbfgs', max_iter=2000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy Score: 0.9574468085106383

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94        38
           1       0.93      1.00      0.97        56

    accuracy                           0.96        94
   macro avg       0.97      0.95      0.95        94
weighted avg       0.96      0.96      0.96        94

